# Tidy Data


> Structuring datasets to facilitate analysis ([Wickham 2014](http://www.jstatsoft.org/v59/i10/paper))

Tidy Data (and the closely related topic of reshaping data) is a way of structuring your data and APIs in a consistent way to make the data analysis process flow more smoothly.

## Rules for the impatient


1. Each variable forms a column
2. Each observation forms a row

In my experience, this is the hardest part of teaching pandas (or R / dplyr / the tidyverse). People can pick up syntax, methods, etc. But knowing what shape your data needs to be in to most easily answer your question is harder.

## NBA Games

Earlier, I grabbed some data from www.basketball-reference.com. It's a tad messy

```
Date,Start (ET),Unnamed: 2,Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 7,Notes
October,,,,,,,,
"Tue, Oct 27, 2015",8:00 pm,Box Score,Detroit Pistons,106.0,Atlanta Hawks,94.0,,
"Tue, Oct 27, 2015",8:00 pm,Box Score,Cleveland Cavaliers,95.0,Chicago Bulls,97.0,,
"Tue, Oct 27, 2015",10:30 pm,Box Score,New Orleans Pelicans,95.0,Golden State Warriors,111.0,,
```

There's some junk in there (what's this `Unnamed: 2` stuff?), but in the raw data, the fields we care about are something like

date       | visitor team    | visitor points | home team     | home points |
---------- | --------------- | -------------- | ------------- | ----------- |
2015-10-07 | Detroit Pistons | 106            | Atlanta Hawks | 94          |

In [ ]:
import pandas as pd

games = pd.read_csv("data/games.csv.gz")
games

We have some general cleaning up to do. We'll use a small method chain to handle all that.

In [ ]:
column_names = {'Date': 'date', 'Start (ET)': 'start',
                'Unamed: 2': 'box', 'Visitor/Neutral': 'away_team', 
                'PTS': 'away_points', 'Home/Neutral': 'home_team',
                'PTS.1': 'home_points', 'Unamed: 7': 'n_ot'}

games = (
    pd.read_csv("data/games.csv.gz")
      .rename(columns=column_names)
      # Drop "blank" rows with >4 missing values
      .dropna(thresh=4)
      # Select just the columns we'll use
      [['date', 'away_team', 'away_points', 'home_team', 'home_points']]
      # Fix the dtype on the `date` column
      .assign(date=lambda x: pd.to_datetime(x['date'], format='%a, %b %d, %Y'))
      # Make a multi-level index with (row_number, date) as the index.
      .set_index('date', append=True)
      .rename_axis(["game_id", "date"])
      .sort_index()
)
games.head()

There's a couple new components in that chain:

1. `DataFrame.rename(columns=dict)` to clean up the names a bit
2. Selecting multiple columns with `[ list_of_columns ]`.
3. `DataFrame.assign` to create a new column from an existing one
4. `set_index` to move a column to the index to be a row label
5. Multi-level row labels. This is like a composite primary key in SQL.


Now onto our question: **How many days of rest did each team have between each game**?

Given that question: is our data tidy?


Or to put it another way: what would a tidy dataset look like? Write down the columns in a tidy dataset for this question.

## Melt


![](images/melt.png)

Our dataset is not tidy because we have *multiple observations per row*. Each row in `games` contains an observation for the home team *and* an observation for the away team.

The `melt` method can help you here. You specify

1. `value_vars`: The colums that the observation is currently spread across
2. The `id_vars`: variables that, together with `value_vars` will uniquely identify the result. These are *repeated* as needed to stay with the original observation.
3. The output column names for each of these.

In [ ]:
teams = (
    games.reset_index()  # Move row labels to columns. melt works just with columns
        .melt(id_vars=['game_id', 'date'],
              value_vars=['away_team', 'home_team'],
              value_name='team', var_name='home_away')
        .sort_values(['game_id', 'date'])
)
teams.head()

Now let's answer the question: How many days of rest did each team get between each game?
Let's start with a single team, the Bulls.

In [ ]:
bulls = teams[teams['team'] == 'Chicago Bulls']
bulls

To answer this for the Bulls, we need two new methods:

1. `.diff()`: Subtract one row from the previous.
2. `.dt`: An *accessor* for getting the components of a datetime / timedelta column.

In [ ]:
bulls['date'].diff(1)

In [ ]:
bulls['date'].diff().dt.days

## Aside: groupby

This pattern of "apply some operation to each group" is common in data analysis. In pandas, we do this with the `.groupby()` method. The components are

1. `DataFrame.groupby(grouper)`: where `grouper` is a column or array to group the dataframe by
2. An operation to apply to each group. This could be an aggregation like `.mean()` or `.count()`, or a 1:1 transformation like `.diff()`, or any arbitrary function passed to `.apply`.

In [ ]:
# previously: bulls['date'].diff().dt.days - 1
# Repeat this for each group with `.groupby()`
# 1. What do we group by?
# 2. What operation do we apply to each group (select date, diff, days, subtract 1)
rest = teams.groupby(...)...
rest

In [ ]:
%load solutions/tidy_groupby_rest.py

Plotting that:

In [ ]:
import seaborn as sns
sns.ecdfplot(rest.dropna());

Let's add that result to the `teams` dataframe.

In [ ]:
teams['rest'] = rest
teams.head()

## Invert the melt

We can use `pandas.pivot_table` to invert the `melt` operation. You describe the output shape you want.

In [ ]:
pd.pivot_table(teams, values='rest', index=['game_id', 'date'], columns='home_away')

In [ ]:
by_game = (pd.pivot_table(teams, values='rest',
                          index=['game_id', 'date'],
                          columns='home_away')
             .rename(columns={'away_team': 'away_rest',
                              'home_team': 'home_rest'})
             .rename_axis(None, axis='columns'))
by_game.dropna().head()

In [ ]:
games_rest = pd.concat([games, by_game], axis='columns').dropna()
games_rest

So now we have two datasets, either of which might be "tidy" *depending on the question*.

When we're asking *team*-level questions (e.g. does the average days of rest vary over the course of the season?) we'd use `teams`. When we're asking *game*-level questions then we'd use `games_rest`.

## Exercise

Does the home team tend to win more often when they're more rested than the away team? Does this question need data on *games* or *teams*?

There's several ways to get at this question. My solution breaks it into three steps:

1. Create a boolean series indicating whether the home team had more rest than the away team
2. Create a boolean series indicating whether the home team won (more points)
3. Group the "home team won" series by the "home team had more rest" series and take the mean.

In [ ]:
# 1. Did the home team have more rest?
home_more_rested = games_rest[...] > games_rest[...]

# 2. Did the home team win?
home_won = games[...] ... 

# 3. Group and take the mean
....groupby(...)....

In [ ]:
# %load solutions/tidy_rest_advantage.py
home_more_rested = games_rest['home_rest'] > games_rest['away_rest']
home_won = games_rest['home_points'] > games_rest['away_points']
home_won.groupby(home_more_rested).mean()


## Next Steps

Next, we'll move to [Performance](Performance.ipynb).